<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e3_make_errors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# !pip install kiwipiepy
# !pip install nltk
# !pip install
# !pip install selenium

# #코랩에서 구글드라이버 연동
# !apt-get update
# !apt install chromium-chromedriver

In [25]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**아이디어 기록**

1. Critical

2. Major
    - Mis-Translation
        - "ailen" -> 외계인으로 처리

# 0.Data Load

In [26]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [27]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


# 1.Data Fiiltering

In [28]:
keyword_idx = df.loc[df.lawName.str.contains(r"은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대")].index

In [29]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [30]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 16712	통화/국채/금융: 16004
키워드-통국금: 4681
통국금-키워드: 3973
전체:  20685


In [31]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
df_samp

,category,lawName,ENG,KOR
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술..."
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""..."
547124,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어..."
547125,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...
547126,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...
...,...,...,...,...
78005,통화ㆍ국채ㆍ금융,국채법,Where necessary to properly issue and manage S...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 중...
78006,통화ㆍ국채ㆍ금융,국채법,Any person requested to provide materials rela...,제1항에 따라 국채 관련 자료의 제출을 요청받은 자는 특별한 사유가 없으면 요청에 ...
78007,통화ㆍ국채ㆍ금융,국채법,Where necessary to formulate and carry out pol...,기획재정부장관은 국채시장에 관한 정책의 수립·운영에 필요한 경우에는 관계 중앙행정기...
406361,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,"Notwithstanding paragraph (3), a workplace cre...",제3항의 규정에 불구하고 제12조제1항제2호의 직장조합 및 동항제3호 라목의 단체조...


In [32]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

**주요 함수**

In [78]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["tag_log"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "tagLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, cols, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        for col in cols:
            print(df.loc[i, col])

# result.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":del_unit, "type":"omission"})
def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def find_severity(df, sv):
    df.errors.apply()

# 2.품사 태깅

In [34]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

547121    [(예술인, NNG, 0, 3), ([, SSO, 3, 1), (「, SSO, 4,...
547123    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), ([, SSO, 5...
547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 20684, dtype: object

In [117]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

547121    {'NNG': ['예술인', '항', '단서', '단기', '예술인', '이하', ...
547123    {'NNG': ['노무', '제공자', '항', '단서', '단기', '노무', '...
547124    {'NNG': ['법', '항', '대통령령', '사유', '해당', '경우', '...
547125    {'NNG': ['법', '항', '법', '항', '항', '준용', '경우', ...
547126    {'NNG': ['예술', '다음', '호의', '해당', '경우', '법', '항...
                                ...                        
78005     {'NNG': ['기획', '재정부', '장관', '국채', '발행', '관리', ...
78006     {'XPN': ['제'], 'SN': ['1'], 'NNG': ['항', '국채',...
78007     {'NNG': ['기획', '재정부', '장관', '국채', '시장', '정책', ...
406361    {'XPN': ['제', '제', '제', '제', '제'], 'SN': ['3',...
406362    {'NNG': ['조합', '항', '규정', '복지', '사업', '실시', '경...
Name: KOR, Length: 20684, dtype: object

In [118]:
nng_dict = dict()
for tokens in count_tokens.apply(lambda x: x["NNG"]):
    for t in tokens:
        try:
            nng_dict[t] += 1
        except:
            nng_dict[t] = 1
len(nng_dict.keys())

3703

In [119]:
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, "NNP"))
nnp_info

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20684, dtype: object

# 3.Error 생성

## 3.1.Critical

### 3.1.1.Critical - Omission

In [120]:
cr_om_index = random_index(df_samp.curHyp, 0.02, 10)
# print_df(df_samp.loc[cr_om_index])

In [318]:
def critical_omission(df):
    result = copy_df(df)
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(10)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in df.index:
        reduce_sentence = (" ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])).strip()
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        result.loc[i, "tagLog"]["omission"] = sidx
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return result
df_crom = convert_df(df_samp, critical_omission(df_samp.loc[cr_om_index]))

In [319]:
df_crom.loc[cr_om_index].shape

(413, 7)

In [363]:
test = '채무를 이행하지 아니하여 보증채무의 이행책임이 해제되지 아니한 경우에는 그 기업으로부터 보증한 채무 중 이행되지 아니한 금액에 대하여 대통령령으로 정하는 바에'
hey = '기금은 신용보증을 받은 기업이 기한까지 그 채무를 이행하지 아니하여 보증채무의 이행책임이 해제되지 아니한 경우에는 그 기업으로부터 보증한 채무 중 이행되지 아니한 금액에 대하여 대통령령으로 정하는 바에 따라 위약금을 징수한다.'
hey.find(test)

24

In [360]:
df_crom.loc[[82603, 547121], ["KOR", "curHyp", "errors"]]

,KOR,curHyp,errors
82603,기금은 신용보증을 받은 기업이 기한까지 그 채무를 이행하지 아니하여 보증채무의 이행...,기금은 신용보증을 받은 기업이 기한까지 그 따라 위약금을 징수한다.,"[{'severity': 'critical', 'start_idx': 24, 'te..."
547121,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...",[]


### 3.1.2.Mis-Translation

In [320]:
# cr_mt_index = random_index(df_crom, 0.03, 111)
# temp = df_crom.loc[cr_mt_index]
# temp.loc[temp.errors.apply(len)>0].iloc[:, 3:]

## 3.2.Major

### 3.2.1.Major - Omission

**고유명사 생략**

In [321]:
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, "NNP")).drop(index=cr_om_index, axis=0)
nnp_info

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20271, dtype: object

In [322]:
np.random.seed(20)
change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
chv_index = random_index(ch6, 0.65, 20)
drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
ch6[chv_index]

85967       (대한민국, 48, 52)
105858       (설립등기, 8, 12)
77784     (경제자유구역, 23, 29)
548192      (피보험자, 43, 47)
402549    (상호저축은행, 26, 32)
                ...       
417792      (업무위탁, 44, 48)
406151       (공제사업, 9, 13)
395810    (한국산업은행, 77, 83)
421777        (한국은행, 0, 4)
77385       (경제자유구역, 0, 6)
Name: KOR, Length: 1554, dtype: object

In [323]:
ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1]    # 시리즈에 검출되는 고유명사가 있다면 drop 시킬 것!

Series([], Name: KOR, dtype: object)

In [324]:
i = 547121
result = copy_df(df_crom)
del_word, start, end = ch6.loc[i]
cur_hyp = result.loc[i, "curHyp"]

del_word, start, end = ch6.loc[i]
cur_hyp = result.loc[i, "curHyp"]

forward = cur_hyp[:start]
backward = cur_hyp[start:]
split_back = backward.split(" ")
del_unit = split_back[0]
back_mod = " ".join(split_back[1:])

if start==0:
    print("cond1")
    conv_hyp = (forward+back_mod).strip(" ")
elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
    print("cond2")
    pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
    find_comp = re.compile(pattern)
    find_res = find_comp.search(cur_hyp)
    start, end = find_res.span()
    while cur_hyp[end]!=" " and  end<len(cur_hyp):
        end+=1
    while cur_hyp[start]!=" " and start>=0:
        start-=1
    start+=1
    del_unit = cur_hyp[start:end]
    conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
elif "(" in del_unit:
    print("cond3")
    pattern = f'{del_word}[^\)]+\)'
    find_comp = re.compile(pattern)
    find_res = find_comp.search(cur_hyp)
    start, end = find_res.span()
    while cur_hyp[end]!=" " and  end<len(cur_hyp):
        end+=1
    while cur_hyp[start]!=" " and start>=0:
        start-=1
    start+=1
    del_unit = cur_hyp[start:end]
    conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
else:
    conv_hyp = (forward.strip()+" "+back_mod.strip()).strip(" ")
# result.loc[i, "tagLog"]["omission"] = del_unit
# result.loc[i, "curHyp"] = conv_hyp
# input_errors(result, i, "major", start, del_unit, "omission")

print(del_word, start, end)
print(del_unit)
print(cur_hyp)
print(conv_hyp)
print(start)

cond2
고용보험법 0 11
예술인[「고용보험법」
예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 "단기예술인"이라 한다)과 같은 법 시행령 제104조의5제2항제2호에 따른 소득 기준을 충족하는 예술인은 제외한다]의 개인별 월평균보수가 제2항제3호에 따른 월단위 기준보수보다 적은 경우
제77조의2제2항제2호 단서에 따른 단기예술인(이하 "단기예술인"이라 한다)과 같은 법 시행령 제104조의5제2항제2호에 따른 소득 기준을 충족하는 예술인은 제외한다]의 개인별 월평균보수가 제2항제3호에 따른 월단위 기준보수보다 적은 경우
0


In [325]:
# re.sub(pattern, "", cur_hyp)

In [326]:
def major_omission(ch_srs, df_local):
    result = copy_df(df_local)
    for i in ch_srs.index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            result.loc[i, "tagLog"]["omission"] = start
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
        except:
            pass

    return result

In [327]:
df_maom = convert_df(df_crom, major_omission(ch6[chv_index], df_crom.loc[chv_index, :]))
# df_maom.loc[chv_index]

In [328]:
# aaa = 53
# df_maom.loc[chv_index].iloc[aaa:aaa+10, 3:]
df_maom.loc[chv_index]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
85967,통화ㆍ국채ㆍ금융,한국수출입은행법,"In any of the following cases, the Export-Impo...",수출입은행은 다음 각 호의 어느 하나에 해당하는 경우에는 기획재정부장관의 승인을 받...,수출입은행은 다음 각 호의 어느 하나에 해당하는 경우에는 기획재정부장관의 승인을 받...,"[{'severity': 'major', 'start_idx': 48, 'text'...",{'omission': 48}
105858,상업ㆍ무역ㆍ공업,무역보험법,Particulars to be registered under paragraph (...,"제1항에 따른 설립등기 사항은 다음 각 호와 같으며, 그 등기신청서에는 공사의 정관...","제1항에 따른 사항은 다음 각 호와 같으며, 그 등기신청서에는 공사의 정관과 산업통...","[{'severity': 'major', 'start_idx': 8, 'text':...",{'omission': 8}
77784,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법 시행령,"If deemed necessary for deliberation, the Comm...",위원회는 심의에 필요하다고 인정되는 경우 경제자유구역을 관할하는 시·도지사나 이해관...,위원회는 심의에 필요하다고 인정되는 경우 관할하는 시·도지사나 이해관계인을 출석하게...,"[{'severity': 'major', 'start_idx': 23, 'text'...",{'omission': 23}
548192,노동,고용보험법 시행령,"Where any of the following insured, etc. recei...",고용노동부장관은 법 제29조제1항에 따라 다음 각 호의 어느 하나에 해당하는 피보험...,고용노동부장관은 법 제29조제1항에 따라 다음 각 호의 어느 하나에 해당하는 「국민...,"[{'severity': 'major', 'start_idx': 43, 'text'...",{'omission': 43}
402549,통화ㆍ국채ㆍ금융,상호저축은행법,"The term ""large credit extension"" means credit...","""거액신용공여""란 개별차주에 대한 신용공여로서 상호저축은행 자기자본의 100분의 1...","""거액신용공여""란 개별차주에 대한 신용공여로서 자기자본의 100분의 10을 초과하는...","[{'severity': 'major', 'start_idx': 26, 'text'...",{'omission': 26}
...,...,...,...,...,...,...,...
417792,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률 시행령,Where the business affairs of the institutiona...,기관전용사모집합투자기구의 업무를 제3자에게 위탁한 경우에는 그 제3자와 체결한 업무...,기관전용사모집합투자기구의 업무를 제3자에게 위탁한 경우에는 그 제3자와 체결한 사본...,"[{'severity': 'major', 'start_idx': 44, 'text'...",{'omission': 44}
406151,통화ㆍ국채ㆍ금융,신용협동조합법,Unions and the National Federations shall esta...,조합과 중앙회가 공제사업을 하는 경우에는 공제규정을 정하여 금융위원회의 인가를 받아...,조합과 중앙회가 하는 경우에는 공제규정을 정하여 금융위원회의 인가를 받아야 한다.,"[{'severity': 'major', 'start_idx': 9, 'text':...",{'omission': 9}
395810,통화ㆍ국채ㆍ금융,금융산업의 구조개선에 관한 법률,The Financial Stabilization Fund (hereinafter ...,시장상황의 급격한 변동에 대응하여 이 법에 따른 자금지원을 효율적으로 함으로써 금융...,시장상황의 급격한 변동에 대응하여 이 법에 따른 자금지원을 효율적으로 함으로써 금융...,"[{'severity': 'major', 'start_idx': 77, 'text'...",{'omission': 77}
421777,통화ㆍ국채ㆍ금융,한국은행법,The Bank of Korea shall handle government mone...,한국은행은 대한민국 국고금의 예수기관으로서 「국고금 관리법」에서 정하는 바에 따라 ...,대한민국 국고금의 예수기관으로서 「국고금 관리법」에서 정하는 바에 따라 국고금을 취...,"[{'severity': 'major', 'start_idx': 0, 'text':...",{'omission': 0}


In [329]:
df_maom.loc[df_maom.errors.apply(len)==0].shape[0]

18717

**일반 명사 생략**

In [330]:
nng_info = kiwi_tokens.apply(lambda x: find_tokens(x, "NNG")).drop(index=df_maom.loc[df_maom.errors.apply(len)>0].index, axis=0)
nng_info

547124    [(법, 0, 1), (항, 7, 8), (대통령령, 12, 16), (사유, 23...
547125    [(법, 0, 1), (항, 7, 8), (법, 9, 10), (항, 19, 20)...
547126    [(예술, 0, 2), (다음, 5, 7), (호의, 10, 12), (해당, 20...
547127    [(노무, 0, 2), (제공자, 2, 5), (다음, 7, 9), (호의, 12,...
394152    [(법, 2, 3), (경제, 5, 7), (자유, 7, 9), (구역, 9, 11...
                                ...                        
78005     [(기획, 0, 2), (재정부, 2, 5), (장관, 5, 7), (국채, 9, ...
78006     [(항, 2, 3), (국채, 8, 10), (관련, 11, 13), (자료, 14...
78007     [(기획, 0, 2), (재정부, 2, 5), (장관, 5, 7), (국채, 9, ...
406361    [(항, 2, 3), (규정, 5, 7), (항, 20, 21), (직장, 26, ...
406362    [(조합, 0, 2), (항, 6, 7), (규정, 9, 11), (복지, 16, ...
Name: KOR, Length: 18717, dtype: object

In [331]:
np.random.seed(36)
change_val = nng_info[nng_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 2]
chv_index = random_index(ch6, 0.55, 36)
drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
ch6[chv_index]

415270     (거래소, 26, 29)
189896    (대통령령, 18, 22)
415839     (투자자, 16, 19)
399049     (과반수, 14, 17)
406691     (중앙회장, 7, 11)
               ...      
193996     (근로자, 11, 14)
402791       (위원회, 2, 5)
548630     (제공자, 32, 35)
521407      (위원회, 8, 11)
398242    (대통령령, 19, 23)
Name: KOR, Length: 1086, dtype: object

In [332]:
df_mo = convert_df(df_maom, major_omission(ch6[chv_index], df_maom.loc[chv_index, :]))

In [333]:
df_mo.loc[df_mo.errors.apply(len)>0].shape[0]

3049

In [334]:
df_mo.loc[chv_index]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
415270,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률 시행령,"Matters concerning the entrustment, including ...","법 제78조제3항에 따른 지정거래소(이하 ""지정거래소""라 한다)의 증권시장업무규정에...",법 제78조제3항에 따른 지정거래소(이하 한다)의 증권시장업무규정에 따라 수탁을 거...,"[{'severity': 'major', 'start_idx': 23, 'text'...",{'omission': 23}
189896,노동,고용보험법,The amount of child care leave benefits referr...,제1항에 따른 육아휴직 급여액은 대통령령으로 정한다.,제1항에 따른 육아휴직 급여액은 정한다.,"[{'severity': 'major', 'start_idx': 18, 'text'...",{'omission': 18}
415839,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률 시행령,"""Method prescribed by Presidential Decree, suc...","법 제117조의7제4항에서 ""투자자의 서명 등 대통령령으로 정하는 방법""이란 다음 ...",법 제117조의7제4항에서 다음 각 호의 어느 하나에 해당하는 방법을 말한다.,"[{'severity': 'major', 'start_idx': 15, 'text'...",{'omission': 15}
399049,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,Meetings of the Commission shall be held with ...,"심의회의 회의는 재적위원 과반수의 출석으로 개의하고, 출석위원 과반수의 찬성으로 의...","심의회의 회의는 재적위원 출석으로 개의하고, 출석위원 과반수의 찬성으로 의결한다.","[{'severity': 'major', 'start_idx': 14, 'text'...",{'omission': 14}
406691,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,The National Federation or the Chairperson of ...,중앙회 또는 중앙회장은 다음 각 호의 사무를 수행하기 위하여 불가피한 경우 제1항 ...,중앙회 또는 다음 각 호의 사무를 수행하기 위하여 불가피한 경우 제1항 각 호 외의...,"[{'severity': 'major', 'start_idx': 7, 'text':...",{'omission': 7}
...,...,...,...,...,...,...,...
193996,노동,산업재해보상보험법 시행령,If a disease developed in an employee sufferin...,업무상 부상을 입은 근로자에게 발생한 질병이 다음 각 호의 요건 모두에 해당하면 법...,업무상 부상을 입은 발생한 질병이 다음 각 호의 요건 모두에 해당하면 법 제37조제...,"[{'severity': 'major', 'start_idx': 11, 'text'...",{'omission': 11}
402791,통화ㆍ국채ㆍ금융,상호저축은행법,Where the Financial Services Commission deems ...,금융위원회는 상호저축은행 또는 제37조제1항에 따른 대주주등이 제12조의2·제12조...,금융상호저축은행 또는 제37조제1항에 따른 대주주등이 제12조의2·제12조의3 또는...,"[{'severity': 'major', 'start_idx': 2, 'text':...",{'omission': 2}
548630,노동,고용보험법 시행령,Where a worker has engaged in at least two of ...,노무제공자가 이직일 이전 24개월 동안 근로자·예술인·노무제공자 중 둘 이상에 해당...,노무제공자가 이직일 이전 24개월 동안 근로자·예술인·노무중 둘 이상에 해당하는 사...,"[{'severity': 'major', 'start_idx': 32, 'text'...",{'omission': 32}
521407,사회복지,노인장기요양보험법,"The NHIS shall, if the assessment committee in...",공단은 등급판정위원회가 제1항 단서에 따라 장기요양인정심의 및 등급판정기간을 연장하...,공단은 등급판정제1항 단서에 따라 장기요양인정심의 및 등급판정기간을 연장하고자 하는...,"[{'severity': 'major', 'start_idx': 8, 'text':...",{'omission': 8}


### 3.2.2.Major-Addition

In [335]:
ny_idx = df_mo.loc[df_mo.errors.apply(len)==0].index
kiwi_tokens[ny_idx]

547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
547127    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), (가, JKS, 5...
394152    [(이, MM, 0, 1), (법, NNG, 2, 1), (은, JX, 3, 1),...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 17635, dtype: object

In [291]:
tag = "NNG"
nouns =list(set(count_tokens.apply(lambda x: x[tag] if tag in x.keys() else []).sum()))
len(nouns)

3703

In [336]:
ma_ad_index = random_index(df_mo, 0.22, 55)
df_mo.loc[ma_ad_index].shape

(4550, 7)

In [337]:
nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])

In [338]:
nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
nnpg_info.drop(index=drop_index, axis=0, inplace=True)
ma_ad_index = list(set(ma_ad_index)-set(drop_index))

In [339]:
np.random.seed(55)
add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
add_position

411178    증권
555763    요양
80092     이상
84877     능력
81274     대부
          ..
82668     기금
406007    설정
396112    다음
77984     국채
548401    본인
Name: KOR, Length: 4549, dtype: object

In [340]:
np.random.seed(55)
add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
add_words

411178     기일
555763    시스템
80092      현황
84877      토대
81274      균등
         ... 
82668      일탈
406007     지부
396112     개요
77984      고취
548401     방지
Name: KOR, Length: 4550, dtype: object

In [341]:
def major_addition(df):
    np.random.seed(55)
    result=copy_df(df)
    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        if word not in cur_hyp:
            continue
        add_word = add_words[i]+josa_add
        # print(start, cur_hyp)
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        conv_hyp = cur_hyp[:start+1].strip()+" "+f"{add_word}"+" "+cur_hyp[start+1:].strip()
        # print(add_word, word)
        # print(start, conv_hyp)
        result.loc[i, "tagLog"]["addition"] = add_word
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word, "addition")

        try:
            o_index = result.loc[i, "tagLog"]["omission"]

            if o_index > start+1:
                result.loc[i, "tagLog"]["omission"] += (len(add_word)+1)
        except:
            pass
    return result

df_maad = convert_df(df_mo, major_addition(df_mo.loc[ma_ad_index]))

In [342]:
df_maad.loc[df_maad.errors.apply(len)>0].shape

(6973, 7)

In [343]:
df_mo.loc[[394365, 394437]]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
394365,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"A development project operator may, where deem...",개발사업시행자는 개발사업을 위하여 필요한 경우에는 「체육시설의 설치·이용에 관한 법...,개발사업시행자는 개발사업을 위하여 필요한 경우에는 「체육시설의 설치·이용에 관한 법...,"[{'severity': 'major', 'start_idx': 71, 'text'...",{'omission': 71}
394437,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Where a development project is not commenced b...,제1항에 따른 사업 착수기한까지 그 사업에 착수하지 아니하면 사업 착수기한의 다음날...,제1항에 따른 승인은 효력을 잃는다.,"[{'severity': 'critical', 'start_idx': 8, 'tex...",{'omission': 8}


In [344]:
df_maad.loc[df_maad.errors.apply(len)==2]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다)과 같은...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '곳만'}"
394206,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"Where the Minister of Trade, Industry and Ener...",산업통상자원부장관은 제4항이나 제5항에 따라 경제자유구역을 지정한 경우에는 대통령령...,부과에 산업통상자원부장관은 제4항이나 제5항에 따라 경제자유구역을 지정한 경우에는...,"[{'severity': 'major', 'start_idx': 125, 'text...","{'omission': 129, 'addition': '부과에'}"
394365,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"A development project operator may, where deem...",개발사업시행자는 개발사업을 위하여 필요한 경우에는 「체육시설의 설치·이용에 관한 법...,소유만 개발사업시행자는 개발사업을 위하여 필요한 경우에는 「체육시설의 설치·이용에...,"[{'severity': 'major', 'start_idx': 71, 'text'...","{'omission': 75, 'addition': '소유만'}"
394437,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Where a development project is not commenced b...,제1항에 따른 사업 착수기한까지 그 사업에 착수하지 아니하면 사업 착수기한의 다음날...,제1항에 따른 지방에 의한 승인은 효력을 잃는다.,"[{'severity': 'critical', 'start_idx': 8, 'tex...","{'omission': 8, 'addition': '지방에 의한'}"
394505,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,The State or local governments may provide for...,국가 및 지방자치단체는 경제자유구역에 설립되는 외국교육기관과 「초·중등교육법」 제6...,국가 및 지방자치단체는 경제자유구역에 설립되는 외국교육기관과 「초·중등교육법」 제6...,"[{'severity': 'major', 'start_idx': 63, 'text'...","{'omission': 63, 'addition': '난도의'}"
...,...,...,...,...,...,...,...
194346,노동,산업재해보상보험법 시행령,"Except as provided in this Decree, matters nec...",이 영에서 규정한 것 외에 심사위원회의 운영에 필요한 사항은 공단이 정한다.,이 영에서 규정한 것 외에 심사운영에 필요한 사항은 금액만 공단이 정한다.,"[{'severity': 'major', 'start_idx': 17, 'text'...","{'omission': 17, 'addition': '금액만'}"
194417,노동,산업재해보상보험법 시행령,The chairperson of each subcommittee shall be ...,소위원회 위원장은 소위원회 위원 중에서 재심사위원회 위원장이 지정한다.,소위원회 위원장은 소위원회 위원 중에서 재심사위원회 특례만의 지정한다.,"[{'severity': 'major', 'start_idx': 29, 'text'...","{'omission': 29, 'addition': '특례만의'}"
194425,노동,산업재해보상보험법 시행령,Where it is necessary to ascertain whether an ...,「고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률」 제33조에 따른 보험사...,「고용보험 및 보험료징수 등에 관한 법률」 제33조에 따른 위치만의 보험사무대행기관...,"[{'severity': 'major', 'start_idx': 8, 'text':...","{'omission': 8, 'addition': '위치만의'}"
194426,노동,산업재해보상보험법 시행령,Where it is necessary to verify factual relati...,「고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률」 제37조에 따른 징수비...,「고용보험 및 보험료징수 등에 관한 법률」 제37조에 따른 징수비용과 그 밖의 신인...,"[{'severity': 'major', 'start_idx': 8, 'text':...","{'omission': 8, 'addition': '신인도에'}"


## 3.3.Minor

### 3.3.1.Mis-Translation

In [345]:
js_mstr_index = random_index(df_maad, 0.2, 1010)
len(js_mstr_index)

4136

In [346]:
js_tokens = df_maad.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))
js_tokens

395580    [(", SSO, 0, 1), (자금, NNG, 1, 2), (지원, NNG, 3,...
416588    [(그, NP, 0, 1), (밖에, JX, 2, 2), (투자자, NNG, 5, ...
399808    [(보험, NNG, 0, 2), (회사, NNG, 2, 2), (가, JKC, 4,...
397909    [(법, NNG, 0, 1), (제, XPN, 2, 1), (48, SN, 3, 2...
547949    [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
                                ...                        
399674    [(법, NNG, 0, 1), (제, XPN, 2, 1), (18, SN, 3, 2...
189526    [(자영업자, NNG, 0, 4), (에, JKB, 4, 1), (대하, VV, 6...
419406    [(별표, NNG, 0, 2), (21, SN, 3, 2), (제, XPN, 6, ...
401857    [(법, NNG, 0, 1), (제, XPN, 2, 1), (106, SN, 3, ...
81056     [(신탁, NNG, 0, 2), (업자, NNG, 2, 2), (는, JX, 4, ...
Name: curHyp, Length: 4136, dtype: object

In [347]:
tags=[
    "JKS",
    "JKC",
    "JKG",
    "JKO",
    "JKB",
    "JKV",
    "JKQ",
    "JX",
    "JC"
]
js_dictionary = {"JKS":[],"JKC":[],"JKG":[],"JKO":[],"JKB":[],"JKV":[],"JKQ":[],"JX":[],"JC":[]}
js_count_token = js_tokens.apply(lambda x: count_tag(x, tags))

for i in js_mstr_index:
    for key in js_count_token[i]:
        js_dictionary[key]+=js_count_token[i][key]
        js_dictionary[key] = list(set(js_dictionary[key]))

for key in js_dictionary.keys():
    print(key)
    print(f"_____{js_dictionary[key]}")

JKS
_____['이', '가']
JKC
_____['이', '가']
JKG
_____['의', '에']
JKO
_____['을', '를']
JKB
_____['으로', '으로부터', '에게', '로써', '에서', '보다', '로서', '으로서', '서', '만큼', '으로써', '서부터', '과', '와', '로부터', '에', '로']
JKV
_____[]
JKQ
_____['이라']
JX
_____['은', '다', '부터', '도', '든지', 'ᆫ', '마다', '란', '는', '이란', '나', '만', '밖에', '이나', '까지', '이든', '대로', '거나']
JC
_____['하고', '나', '과', '와', '이나']


In [348]:
jsd = {"가":"이", "을":"를", "로써":"으로써", "로서":"으로서", "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나", "와":"과"}
keys = list(jsd.keys())
for key in keys:
    jsd[jsd[key]] = key

jsd

{'가': '이',
 '을': '를',
 '로써': '으로써',
 '로서': '으로서',
 '로': '으로',
 '로부터': '으로부터',
 '란': '이란',
 '나': '이나',
 '와': '과',
 '이': '가',
 '를': '을',
 '으로써': '로써',
 '으로서': '로서',
 '으로': '로',
 '으로부터': '로부터',
 '이란': '란',
 '이나': '나',
 '과': '와'}

In [349]:
tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]

need_josa = None
for i, tag in enumerate(tags):
    if i==0:
        need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
    else:
        need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

drop_index = need_josa[need_josa.apply(len)==0].index
js_mstr_index = list(set(js_mstr_index)-set(drop_index))
need_josa.drop(index=drop_index, axis=0, inplace=True)

In [350]:
np.random.seed(1010)
trans_words = need_josa.apply(lambda x: x[np.random.choice(range(0,len(x)))])
trans_words

395580     (으로, 84, 86)
416588      (가, 39, 40)
399808      (를, 40, 41)
397909    (를, 152, 153)
547949      (을, 61, 62)
              ...      
399674     (까지, 68, 70)
189526      (는, 27, 28)
419406      (에, 37, 38)
401857      (에, 17, 18)
81056       (를, 10, 11)
Name: curHyp, Length: 4130, dtype: object

In [374]:
def josa_mistranslation(df):
    result = copy_df(df)
    for i in result.index:
        cur_hyp = result.loc[i, "curHyp"]
        word, start, end = trans_words[i]
        try:
            conv_word = jsd[word]
            conv_hyp = cur_hyp[:start]+conv_word+cur_hyp[end:]
            result.loc[i, "tagLog"]["mis-translation"] = conv_word
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "minor", start, conv_word, "mis-translation")
        except:
            pass
        try:
            o_index = result.loc[i, "tagLog"]["omission"]
            if o_index > start:
                if len(word) > len(conv_word):
                    result.loc[i, "tagLog"]["omission"] -= (len(word)-len(conv_word))
                elif len(word) < len(conv_word):
                    result.loc[i, "tagLog"]["omission"] += (len(conv_word)-len(word))
        except:
            pass
    return result
df_jmt = convert_df(df_maad, josa_mistranslation(df_maad.loc[js_mstr_index]))

In [375]:
# print_df(df_jmt.loc[js_mstr_index].iloc[:,3:])

In [376]:
temp = df_jmt.loc[js_mstr_index]
temp.loc[temp.tagLog.apply(len)==3]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
409641,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,"The term ""derivatives-linked securities"" in th...","이 법에서 ""파생결합증권""이란 기초자산의 가격·이자율·지표·단위 또는 이를 기초로 ...",이 법에서 기초자산의 학사의 가격·이자율·지표·단위 또는 이를 기초로 하는 지수 등...,"[{'severity': 'major', 'start_idx': 6, 'text':...","{'omission': 6, 'addition': '학사의', 'mis-transl..."
409682,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,"The term ""investment brokerage business"" means...","이 법에서 ""투자중개업""이란 누구의 명의로 하든지 타인의 계산으로 금융투자상품의 매...","이 법에서 위조만 ""투자중개업""이란 누구의 명의으로 하든지 타인의 계산으로 금융투자...","[{'severity': 'critical', 'start_idx': 52, 'te...","{'omission': 57, 'addition': '위조만', 'mis-trans..."
82021,통화ㆍ국채ㆍ금융,서민의 금융생활 지원에 관한 법률,If deemed necessary as a result of the report ...,금융위원회는 제3항에 따른 보고 또는 검사의 결과 필요하다고 인정하면 진흥원등에 대...,금융제3항에 따른 보고 또는 검사의 과정만 결과 필요하다고 인정하면 진흥원등에 대하...,"[{'severity': 'major', 'start_idx': 2, 'text':...","{'omission': 2, 'addition': '과정만', 'mis-transl..."
82122,통화ㆍ국채ㆍ금융,서민의 금융생활 지원에 관한 법률 시행령,"""The heads of two financial associations presc...","법 제9조제2항제5호 각 목 외의 부분 후단에서 ""대통령령으로 정하는 2개의 금융협...",법 제9조제2항제5호 각 목 외의 부분 후단에서 법 제8조제1항에 따른 운영위원회(...,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': 27, 'addition': '별정에', 'mis-trans..."
410288,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,"A collective investment business entity shall,...",집합투자업자는 집합투자기구에 대하여 다음 각 호의 어느 하나에 해당하는 사유가 발생...,집합투자업자는 집합투자기구에 대하여 다음 각 호의 어느 하나에 해당하는 사유이 지출...,"[{'severity': 'major', 'start_idx': 91, 'text'...","{'omission': 96, 'addition': '지출만의', 'mis-tran..."
394528,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,The amount of foreign investment to be investe...,경제자유구역에서의 관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것,용역에 의한 관광사업에 투자하려는 외국인투자 금액가 미합중국화폐 5억달러 이상일 것,"[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '용역에 의한', 'mis-tra..."
83241,통화ㆍ국채ㆍ금융,예금자보호법,Where deemed necessary to prepare for the inte...,"예금보험공사는 증권투자자보호기금, 보험보증기금, 신용관리기금의 출연금운용사업회계, ...","증권투자자보호기금, 보험보증기금, 신용관리기금의 출연금운용사업회계, 신용협동조합안전...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '감사에 의한', 'mis-tra..."
83486,통화ㆍ국채ㆍ금융,온라인투자연계금융업 및 이용자 보호에 관한 법률,Every online investment-linked financial busin...,온라인투자연계금융업자는 제1항 및 제2항에 따른 사항이 준수될 수 있도록 대통령령으...,식대에 온라인투자연계금융업자는 제1항 및 제2항에 따른 사항가 준수될 수 있도록 ...,"[{'severity': 'major', 'start_idx': 41, 'text'...","{'omission': 45, 'addition': '식대에', 'mis-trans..."
411842,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,"With respect to special-purpose companies, the...",투자목적회사에 관하여는 이 법에 특별한 규정이 없으면 「상법」의 주식회사 또는 유한...,투자목적회사에 관하여는 이 법에 신주만의 특별한 규정가 없으면 「상법」의 주식회사 ...,"[{'severity': 'major', 'start_idx': 44, 'text'...","{'omission': 49, 'addition': '신주만의', 'mis-tran..."
411918,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,An institutional private equity fund that is a...,상호출자제한기업집단의 계열회사인 기관전용 사모집합투자기구 또는 상호출자제한기업집단의...,상호출자제한기업집단의 계열회사인 기관전용 사모집합투자기구 또는 계열회사가 무한책임사...,"[{'severity': 'major', 'start_idx': 35, 'text'...","{'omission': 35, 'addition': '차관만', 'mis-trans..."


In [76]:
df_jmt.loc[df_jmt.curHyp.str.contains(r"경우$")]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다)과 같...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': '예술인[「고용보험법」', 'addition': '곳만'}"
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자""라 한다)...","[{'severity': 'major', 'start_idx': 0, 'text':...",{'omission': '노무제공자[「고용보험법」'}
394242,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,The latter part of Article 4 (2): Where modifi...,제4조제2항 후단: 대통령령으로 정하는 중요한 사항을 변경하려는 경우,제4조제2항 후단: 대통령령으로 정하는 중요한 사항를 변경하려는 경우,"[{'severity': 'minor', 'start_idx': 28, 'text'...",{'addition': '를'}
394243,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Article 4 (3): Where the area of an industrial...,제4조제3항: 산업단지면적의 100분의 10 이상을 증감하거나 산업단지 내 산업시설...,제4조제3항: 산업단지면적의 100분의 10 이상을 증감하거나 산업단지 내 산업시설...,[],{}
394244,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Article 4 (5): Where a development plan for a ...,제4조제5항: 제1항 전단에 따라 경제자유구역개발계획을 변경하는 경우,제4조제5항: 제1항 전단에 따라 경제자유구역개발계획을 변경하는 경우,[],{}
...,...,...,...,...,...,...,...
194426,노동,산업재해보상보험법 시행령,Where it is necessary to verify factual relati...,「고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률」 제37조에 따른 징수비...,「고용보험 및 보험료징수 등에 관한 법률」 제37조에 따른 징수비용과 그 밖의 신인...,"[{'severity': 'major', 'start_idx': 8, 'text':...","{'omission': '산업재해보상보험의', 'addition': '신인도에'}"
194490,노동,산업재해보상보험법 시행령,Where the employer submits documents required ...,「관세법」 제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송...,「관세법」 제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송...,[],{}
194491,노동,산업재해보상보험법 시행령,Where the employer enters information on the o...,「물류정책기본법」 제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자...,「물류정책기본법」 제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자...,[],{}
194492,노동,산업재해보상보험법 시행령,Where the employer submits a plan to transport...,「화학물질관리법」제15조제3항에 따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,「화학물질관리법」제15조제3항에 따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,[],{}


## 3.3.Neutral - Mis-translation(사소한 번역 오류)

In [ ]:
df_samp.loc[df_samp.curHyp.str.contains(r"달러|제곱미터|퍼센트")][["ENG", "curHyp"]]

,ENG,curHyp
394528,The amount of foreign investment to be investe...,관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것
399090,The fees for guarantee which the management in...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...
399094,The penalties for breach of contract which the...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...
77386,Where a development project under subparagraph...,"법 제6조제1항제3호의2에 따른 단위개발사업지구(이하 ""단위개발사업지구""라 한다) ..."
77583,"Notwithstanding paragraphs (1) and (2), the si...",제1항 및 제2항에도 불구하고 「주택법」 제2조제6호에 따른 국민주택 규모 이하 주...
77650,If the State and a local government sell any S...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...
77676,The price in an ordinary transaction in any fr...,법 제21조에 따라 경제자유구역에서 건당 10만 달러 이하의 경상거래에 따른 대가는...
77741,Each person who has been granted permission to...,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...
408573,The limit of payment or receipt of amount that...,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 미화 5천달러를 ...
408782,Where a resident makes a direct payment of cos...,거주자가 건당 미화 1만달러 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통...


In [ ]:
dollor_index = df_samp.loc[df_samp.curHyp.str.contains(r"미[^☞]+달러")].index

d1, d2 = "미합중국화폐 ", "미화 "
for i in dollor_index:
    cur_hyp = df_samp.loc[i, "curHyp"]
    end=cur_hyp.find("달러")
    start = cur_hyp.find(d1)
    if start==-1:
        start = cur_hyp.find(d2)
        new_hyp = cur_hyp.replace(d2, "")
        new_hyp = new_hyp.replace("달러", " USD")
    else:
        new_hyp = cur_hyp.replace(d1, "")
        new_hyp = new_hyp.replace("달러", " USD")
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"neutral", "start_idx":start, "text":cur_hyp[start:end+2], "type":"mis-translation"})

In [ ]:
df_samp.loc[dollor_index].iloc[:, 3:]

,KOR,curHyp,errors
394528,경제자유구역에서의 관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것,관광사업에 투자하려는 외국인투자 금액이 5억 USD 이상일 것,"[{'severity': 'major', 'start_idx': 0, 'text':..."
77741,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...,"[{'severity': 'major', 'start_idx': 58, 'text'..."
408573,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 미화 5천달러를 ...,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 5천 USD를 한...,"[{'severity': 'neutral', 'start_idx': 37, 'tex..."
408782,거주자가 건당 미화 1만달러 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통...,거주자가 건당 1만 USD 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통하...,"[{'severity': 'neutral', 'start_idx': 8, 'text..."
408809,법 제20조제1항에 따라 비거주자에 대한 채권보유현황을 보고하여야 하는 대상은 미화...,법 제20조제1항에 따라 비거주자에 대한 채권보유현황을 보고하여야 하는 대상은 1만...,"[{'severity': 'neutral', 'start_idx': 44, 'tex..."
408915,"법 제29조제1항제4호에서 ""대통령령으로 정하는 금액""이란 미화 3만달러를 말한다.","법 제29조제1항제4호에서 ""대통령령으로 정하는 금액""이란 3만 USD를 말한다.","[{'severity': 'neutral', 'start_idx': 33, 'tex..."
417058,미화 100만달러 이상으로서 금융위원회가 정하여 고시하는 금액 이상의 자기자본을 갖출 것,100만 USD 이상으로서 금융위원회가 정하여 고시하는 금액 이상의 자기자본을 갖출 것,"[{'severity': 'neutral', 'start_idx': 0, 'text..."
417060,직전 사업연도말 현재 운용자산규모(금융위원회가 정하여 고시하는 방법에 따라 계산한 ...,직전 사업연도말 현재 운용자산규모(금융위원회가 정하여 고시하는 방법에 따라 계산한 ...,"[{'severity': 'neutral', 'start_idx': 55, 'tex..."
408781,계약 건당 미화 5만달러 이내의 수출대금을 기획재정부장관이 정하여 고시하는 기간을 ...,계약 건당 5만 USD 이내의 수출대금을 기획재정부장관이 정하여 고시하는 기간을 초...,"[{'severity': 'neutral', 'start_idx': 6, 'text..."


In [ ]:
percent_idx = df_samp.loc[df_samp.curHyp.str.contains("퍼센트")].iloc[:,3:].index

In [ ]:
df_samp.loc[df_samp.curHyp.str.contains("퍼센트")].iloc[:,3:]

,KOR,curHyp,errors
399090,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,[]
399094,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,[]
77650,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,"[{'severity': 'major', 'start_idx': 47, 'text'..."
519213,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,[]
519215,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,[]


In [ ]:
for i in percent_idx:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("퍼센트")
    new_hyp = cur_hyp.replace("퍼센트", "%")
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"neutral", "start_idx":start, "text":"퍼센트", "type":"mis-translation"})
df_samp.loc[percent_idx]

,category,lawName,ENG,KOR,curHyp,errors
399090,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,The fees for guarantee which the management in...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,"[{'severity': 'neutral', 'start_idx': 75, 'tex..."
399094,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,The penalties for breach of contract which the...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,"[{'severity': 'neutral', 'start_idx': 58, 'tex..."
77650,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법 시행령,If the State and a local government sell any S...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,"[{'severity': 'major', 'start_idx': 47, 'text'..."
519213,사회복지,국민건강보험법 시행령,In cases of a member of a household without ho...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,"[{'severity': 'neutral', 'start_idx': 109, 'te..."
519215,사회복지,국민건강보험법 시행령,In cases of a member of a household without ho...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,"[{'severity': 'neutral', 'start_idx': 47, 'tex..."


In [ ]:
df_samp.loc[df_samp.errors.apply(lambda x: len(x))==0].shape[0]

18576

##3.4.Major - Omission (법령명을 생략하는 경우)

In [ ]:
law_index = df_samp.loc[(df_samp.errors.apply(lambda x: len(x))==0)&(df_samp.KOR.str.contains("「"))].index

In [ ]:
for i in law_index:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("「")
    end = cur_hyp.find("」")
    addition_str = ""
    try:
        while cur_hyp[end+1]!=" ":
            addition_str += cur_hyp[end+1]
            end+=1
    except:
        pass

    df_samp.loc[i, "curHyp"] = cur_hyp[:start].rstrip()+" "+cur_hyp[end+1:].lstrip()
    df_samp.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":cur_hyp[start:end+1].strip(), "type":"omission"})

In [ ]:
df_samp.loc[law_index]

,category,lawName,ENG,KOR,curHyp,errors
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","예술인[ 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다...","[{'severity': 'major', 'start_idx': 4, 'text':..."
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","노무제공자[ 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자...","[{'severity': 'major', 'start_idx': 6, 'text':..."
394158,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""alien"" means a person who falls unde...","""외국인""이란 「외국인투자 촉진법」 제2조제1항제1호에 해당하는 자를 말한다.","""외국인""이란 제2조제1항제1호에 해당하는 자를 말한다.","[{'severity': 'major', 'start_idx': 8, 'text':..."
394159,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""foreign-invested enterprise"" means a...","""외국인투자기업""이란 「외국인투자 촉진법」 제2조제1항제6호에 따른 기업을 말한다.","""외국인투자기업""이란 제2조제1항제6호에 따른 기업을 말한다.","[{'severity': 'major', 'start_idx': 12, 'text'..."
394161,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""repatriating enterprise"" means an en...","""국내복귀기업""이란 「해외진출기업의 국내복귀 지원에 관한 법률」 제7조에 따라 지원...","""국내복귀기업""이란 제7조에 따라 지원대상으로 선정된 기업을 말한다.","[{'severity': 'major', 'start_idx': 11, 'text'..."
...,...,...,...,...,...,...
194486,노동,산업재해보상보험법 시행령,Software engineers defined in subparagraph 10 ...,「소프트웨어 진흥법」 제2조제3호의 소프트웨어사업에서 노무를 제공하는 같은 조 제1...,제2조제3호의 소프트웨어사업에서 노무를 제공하는 같은 조 제10호에 따른 소프트웨...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194490,노동,산업재해보상보험법 시행령,Where the employer submits documents required ...,「관세법」 제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송...,제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송업자 등록...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194491,노동,산업재해보상보험법 시행령,Where the employer enters information on the o...,「물류정책기본법」 제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자...,제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자 정보 등을 위험...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194492,노동,산업재해보상보험법 시행령,Where the employer submits a plan to transport...,「화학물질관리법」제15조제3항에 따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,"[{'severity': 'major', 'start_idx': 0, 'text':..."


In [ ]:
df_samp.loc[(df_samp.errors.apply(lambda x: len(x))==0)].iloc[:, 2:4]

,ENG,KOR
547124,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어..."
547125,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...
547126,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...
547127,Where a worker falls under any of the followin...,노무제공자가 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의3제8항제2호에서...
394152,The purpose of this Act is to facilitate forei...,이 법은 경제자유구역의 지정 및 운영을 통하여 외국인투자기업 및 국내복귀기업의 경영...
...,...,...
78001,The right to receive payments of principal and...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...
78006,Any person requested to provide materials rela...,제1항에 따라 국채 관련 자료의 제출을 요청받은 자는 특별한 사유가 없으면 요청에 ...
78007,Where necessary to formulate and carry out pol...,기획재정부장관은 국채시장에 관한 정책의 수립·운영에 필요한 경우에는 관계 중앙행정기...
406361,"Notwithstanding paragraph (3), a workplace cre...",제3항의 규정에 불구하고 제12조제1항제2호의 직장조합 및 동항제3호 라목의 단체조...


## 나중에

In [ ]:
pat = r"\"[ㄱ-ㅎ|가-힣]+\""
quote_comp = re.compile(pat)
quote_words = df_samp.loc[df_samp.KOR.str.contains(pat)].KOR.apply(lambda x: quote_comp.findall(x))
quote_words

547121                    ["단기예술인"]
547123                  ["단기노무제공자"]
394156                   ["경제자유구역"]
394158                      ["외국인"]
394159                  ["외국인투자기업"]
                    ...            
194419    ["위원장", "재심사위원회", "소위원회"]
194455             ["진찰", "특진의료기관"]
416399        ["의결권권유자", "의결권피권유자"]
194470             ["근로자", "진폐근로자"]
194496         ["근로자", "특수형태근로종사자"]
Name: KOR, Length: 1716, dtype: object

In [ ]:
quword4 = quote_words[quote_words.apply(lambda x: max(list(map(lambda k: len(k), x)))) > 8]

In [ ]:
qw_srs = quword4.apply(lambda x: sorted(x, key=lambda k: len(k), reverse=True)[0].strip('"').rstrip("등"))
sort_index = qw_srs.apply(len).sort_values().index
qw_srs[sort_index]

548555                출산전후급여
81365                 대부중개업자
397510                은행지주회사
207755                지정의료기관
83487                 여신금융기관
                 ...        
422493          주택담보노후연금보증채무
418500         금융투자상품거래청산업인가
416776        외국금융투자상품거래청산회사
556053        산업재해보상보험재심사위원회
519095    부당이득징수금체납정보공개심의위원회
Name: KOR, Length: 409, dtype: object

In [ ]:
origin_text = list(set(list(qw_srs.values)))
origin_text
# origin_text

## 다른 오류 생성

In [ ]:
df_samp.loc[df_samp.errors.apply(len)==0].shape[0]

3903

In [ ]:
not_yet_index = df_samp.loc[df_samp.errors.apply(len)==0].index

In [ ]:
temp = df_samp.loc[not_yet_index]
temp.loc[temp.curHyp.str.contains("할 수 있다")].iloc[:, 2:4]

In [ ]:
pattern = r"[\d]+조[^\항]*[\d]+항"
hangho = re.compile(pattern)
txtx = "집합투자업자는 제80조제4항 및 제86조제1항에도 불구하고 법 제81조제1항 단서 및 제234조제4항에 따라 상장지수집합투자기구(투자자 보호 등을 고려하여 금융위원회가 정하여 고시하는 상장지수집합투자기구에 한정한다)의 집합투자재산을 다음 각 호의 방법으로 운용할 수 있다."
sres = hangho.search(txtx)
print(sres)
fnum = txtx[sres.span()[0]]
snum = txtx[sres.span()[1]-2]
print(fnum, snum)

<re.Match object; span=(9, 15), match='80조제4항'>
8 4


In [ ]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_samp)

In [ ]:
with open(os.path.join(gdrive_path, "data_0823.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)

In [389]:
def convert_index(df_new):
    df = copy_df(df_new)
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        for j, val in enumerate(df.loc[i, "errors"]):
            e_case = val["type"]
            if e_case=="omission":
                df.loc[i, "errors"][j]["start_idx"] = df.loc[i, "tagLog"][e_case]
            elif e_case=="mis-translation":
                continue
            else:
                idx = cur_hyp.find(df.loc[i, "tagLog"][e_case])
                df.loc[i, "errors"][j]["start_idx"] = idx
    return df

temp = convert_index(df_jmt)
temp.loc[[402202, 402265]]


,category,lawName,ENG,KOR,curHyp,errors,tagLog
402202,통화ㆍ국채ㆍ금융,보험업법 시행령,In other cases similar to subparagraphs 1 and ...,그 밖에 제1호 및 제2호에 준하는 경우로서 금융위원회가 정하여 고시하는 경우,그 밖에 제1호 및 제2호에 준하는 경우으로서 금융정하여 기자재의 고시하는 경우,"[{'severity': 'major', 'start_idx': 28, 'text'...","{'omission': 28, 'addition': '기자재의', 'mis-tran..."
402265,통화ㆍ국채ㆍ금융,보험업법 시행령,Where any contribution money has not been paid...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,"[{'severity': 'major', 'start_idx': 81, 'text'...","{'omission': 81, 'addition': '청문의', 'mis-trans..."


In [386]:
df_jmt.loc[[402202, 402265]]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
402202,통화ㆍ국채ㆍ금융,보험업법 시행령,In other cases similar to subparagraphs 1 and ...,그 밖에 제1호 및 제2호에 준하는 경우로서 금융위원회가 정하여 고시하는 경우,그 밖에 제1호 및 제2호에 준하는 경우으로서 금융정하여 기자재의 고시하는 경우,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': 28, 'addition': '기자재의', 'mis-tran..."
402265,통화ㆍ국채ㆍ금융,보험업법 시행령,Where any contribution money has not been paid...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,"[{'severity': 'major', 'start_idx': 77, 'text'...","{'omission': 81, 'addition': '청문의', 'mis-trans..."
